In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
df = pd.read_csv(r'DATA/AMES_Final_DF.csv')
df.head()

,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,...,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD,Sale Condition_AdjLand,Sale Condition_Alloca,Sale Condition_Family,Sale Condition_Normal,Sale Condition_Partial
0,141.0,31770,6,5,1960,1960,112.0,639.0,0.0,441.0,...,0,0,0,0,1,0,0,0,1,0
1,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,270.0,...,0,0,0,0,1,0,0,0,1,0
2,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,406.0,...,0,0,0,0,1,0,0,0,1,0
3,93.0,11160,7,5,1968,1968,0.0,1065.0,0.0,1045.0,...,0,0,0,0,1,0,0,0,1,0
4,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,137.0,...,0,0,0,0,1,0,0,0,1,0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2925 entries, 0 to 2924
Columns: 274 entries, Lot Frontage to Sale Condition_Partial
dtypes: float64(11), int64(263)
memory usage: 6.1 MB


### STEP 1: Label we are trying to predict is the SalePrice Column. Separate out data into X features and y labels

In [9]:
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

### STEP 2: Split up into Training Set and Test Set. Set Test proportion to 10% and specify random state to 101

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1, random_state=101)

### STEP 3: dataset features has variety of scales and units. Scale the X features.

In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
scaled_X_train = scaler.transform(X_train)
scaled_X_test = scaler.transform(X_test)

### STEP 4: REGULARIZATION using ElasticNet

In [11]:
from sklearn.linear_model import ElasticNet

base_elastic_model = ElasticNet()

### STEP 5: Adjust hyperparameters

In [12]:
from sklearn.model_selection import GridSearchCV

param_grid = {'alpha':[0.1,1,5,10,100], 'l1_ratio':[.1,.7,.99,1]}
grid_model = GridSearchCV(base_elastic_model,
                         param_grid=param_grid,
                         scoring='neg_mean_squared_error',
                         cv=5,verbose=1)

In [13]:
grid_model.fit(scaled_X_train,y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


/Users/chestergarettcalingacion/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 354244712120.8133, tolerance: 1355206692.5276783
  model = cd_fast.enet_coordinate_descent(
/Users/chestergarettcalingacion/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 363490863623.81683, tolerance: 1307913805.6588454
  model = cd_fast.enet_coordinate_descent(
/Users/chestergarettcalingacion/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 408539308800.7896, tolerance: 1415056940.006107
  model = cd_fast.enet_coordinate_descent(
/Users/cheste

GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid={'alpha': [0.1, 1, 5, 10, 100],
                         'l1_ratio': [0.1, 0.7, 0.99, 1]},
             scoring='neg_mean_squared_error', verbose=1)

In [14]:
grid_model.best_params_

{'alpha': 100, 'l1_ratio': 1}

### STEP 6: EVALUATE MODEL'S PERFORMANCE

In [16]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred = grid_model.predict(scaled_X_test)
mean_absolute_error(y_test,y_pred)

14195.354900562173

In [17]:
np.sqrt(mean_squared_error(y_test,y_pred))

20558.50856689316

In [18]:
np.mean(df['SalePrice'])

180815.53743589742

#### compare the mean of actual y vs mean absolute error and RMSE